In [1]:
import submitit
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run single structure

In [2]:
f = Path("../structures/misc/ts4_test.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

2025-07-28 10:24:47 INFO  frust.stepper: Working dir: .


In [3]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=10,
    mem_gb=30,
):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "DFT", {
        "wB97X-D3": None,
        "6-31G**": None,
        "TightSCF": None,
        "SlowConv": None,
        "OptTS":    None,
        "Freq":    None,
        "NoSym":   None,
    })
    
    # detailed_inp = """%CPCM\nSMD TRUE\nSMDSOLVENT "chloroform"\nend"""
    # df2 = step.orca(df1, "DFT-SP", {
    #     "wB97X-D3": None,
    #     "6-31+G**": None,
    #     "TightSCF": None,
    #     "SP":       None,
    #     "NoSym":    None,
    # },detailed_inp)

    df1.to_parquet(f"{results_dir}/results_{name}.parquet")

In [4]:
DEBUG           = False
N_CORES         = 25
MEM_GB          = 165
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "ts4_TMP_opt"

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: ts4_test
